from example https://github.com/reichenbch/RAG-examples/blob/main/LangChain%20LLamaIndex%20RAG.ipynb

In [1]:
import sys
sys.path.insert(0, '/home/sulcan/Documents/ipac-logbook/code/')

In [2]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.vectorstores import FAISS
from hugginface_wrapper import HuggingFaceLLM
from langchain.chains.question_answering import load_qa_chain
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import pandas as pd

model_id = "mistralai/Mistral-7B-Instruct-v0.2" # 'microsoft/phi-1.5'
device = 'cuda'
quantization_config = None # BitsAndBytesConfig(load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = model_id, 
                                             device_map = device,
                                             quantization_config = quantization_config)

df = pd.read_csv('/home/sulcan/Documents/malapa/data/abstracts.csv')
abstracts = list(df['Content'])

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
llm = HuggingFaceLLM(model_data = {'tokenizer' : tokenizer, 'model' : model}, max_new_tokens = 1024)

In [4]:
language = "English"
context = "{context}"

template =  """I want you to act as a question answering bot which uses the context mentioned and answer in
a concise manner and doesn't make stuff up.
You will answer question based on the context - """ + str(context) + """. 
You will create content in """ + str(language) + """ language.
Question: {question}
Answer:"""
print(template)
QA_CHAIN_PROMPT = PromptTemplate(input_variables = ['question', 'context'], template = template)

I want you to act as a question answering bot which uses the context mentioned and answer in
a concise manner and doesn't make stuff up.
You will answer question based on the context - {context}. 
You will create content in English language.
Question: {question}
Answer:


In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") 
vector_store = FAISS.from_texts(abstracts, embeddings)
vector_store.save_local("../data/faiss_doc_idx")

In [6]:
chain = load_qa_chain(llm, chain_type="refine")

In [7]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vector_store.as_retriever(), chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

In [8]:
result = qa_chain({"query": "Give me a list of submissions that talk about thoughts."})

/home/sulcan/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [9]:
print(result['result'])

I want you to act as a question answering bot which uses the context mentioned and answer in
a concise manner and doesn't make stuff up.
You will answer question based on the context - In this paper, we show a textual analysis of past ICALEPCS and IPAC conference proceedings to gain insights into the research trends and topics discussed in the field. We use natural language processing techniques to extract meaningful information from the abstracts and papers of past conference proceedings. We extract topics to visualize and identify trends, analyze their evolution to identify emerging research directions, and highlight interesting publications based solely on their content with an analysis of their network. Additionally, we will provide an advanced search tool to better search the existing papers to prevent duplication and easier reference findings. Our analysis provides a comprehensive overview of the research landscape in the field and helps researchers and practitioners to better un